# DQN algrorithm with BigDL

In [11]:
import random
import gym
import numpy as np
from collections import deque

from bigdl.dataset.transformer import Sample
from bigdl.nn.layer import *
from bigdl.util.common import *
from bigdl.optim.optimizer import *
from bigdl.nn.criterion import *

init_engine()

In [16]:
class DQNAgent:
    def __init__(self, state_size, action_size, gamma=0.95, epsilon=1.0,
                 epsilon_min=0.01, epsilon_decay=0.995, learning_rate=0.001):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000)
        self.gamma = gamma  # discount rate
        self.epsilon = epsilon  # exploration rate
        self.epsilon_min = epsilon_min
        self.epsilon_decay = epsilon_decay
        self.learning_rate = learning_rate
        self.model = self._build_model()

    def _build_model(self):
        # Neural Net for Deep-Q learning Model
        model = Sequential()

        model.add(Linear(self.state_size, 24))
        model.add(ReLU())

        model.add(Linear(24, 24))
        model.add(ReLU())

        model.add(Linear(24, self.action_size))
        return model

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.forward(state)
        return np.argmax(act_values[0])  # returns action

    def replay(self, batch_size):
        minibatch = random.sample(self.memory, batch_size)
        data_batch = []
        target_batch = []
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma *
                          np.amax(self.model.forward(next_state)[0]))
            target_f = self.model.forward(state)
            target_f[0][action] = target

            data_batch.append(state[0])
            target_batch.append(target_f)
        batch_rdd = sc.parallelize(data_batch).zip(sc.parallelize(target_batch)).map(
            lambda x: Sample.from_ndarray(x[0], x[1]))
        
        self.model = Optimizer(model=self.model,
                              training_rdd=batch_rdd,
                              criterion=MSECriterion(),
                              optim_method=Adam(learningrate=self.learning_rate),
                              end_trigger=MaxEpoch(1),
                              batch_size=2).optimize()
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay


In [17]:
EPISODES=2000
sc = SparkContext.getOrCreate(conf=create_spark_conf())

if __name__ == "__main__":
    env = gym.make('CartPole-v1')
    state_size = env.observation_space.shape[0]
    action_size = env.action_space.n
    agent = DQNAgent(state_size, action_size)
    # agent.load("./save/cartpole-dqn.h5")
    done = False
    batch_size = 64

    for e in range(EPISODES):
        state = env.reset()
        state = np.reshape(state, [1, state_size])
        for time in range(500):
            #env.render()
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            reward = reward if not done else -100
            next_state = np.reshape(next_state, [1, state_size])
            agent.remember(state, action, reward, next_state, done)
            state = next_state
            if done or time==498:
                print("episode: {}/{}, score: {}, e: {:.2}"
                      .format(e, EPISODES, time, agent.epsilon))
                break
        if len(agent.memory) > batch_size:
            agent.replay(batch_size)
        # if e % 10 == 0:
        #     agent.save("./save/cartpole-dqn.h5")


[2017-09-04 14:52:56,253] Making new env: CartPole-v1


creating: createSequential
creating: createLinear
creating: createReLU
creating: createLinear
creating: createReLU
creating: createLinear
episode: 0/2000, score: 17, e: 1.0
episode: 1/2000, score: 16, e: 1.0
episode: 2/2000, score: 14, e: 1.0
episode: 3/2000, score: 20, e: 1.0
creating: createMSECriterion
creating: createAdam
creating: createMaxEpoch
creating: createOptimizer
episode: 4/2000, score: 17, e: 0.99
creating: createMSECriterion
creating: createAdam
creating: createMaxEpoch
creating: createOptimizer
episode: 5/2000, score: 22, e: 0.99
creating: createMSECriterion
creating: createAdam
creating: createMaxEpoch
creating: createOptimizer
episode: 6/2000, score: 20, e: 0.99
creating: createMSECriterion
creating: createAdam
creating: createMaxEpoch
creating: createOptimizer
episode: 7/2000, score: 20, e: 0.98
creating: createMSECriterion
creating: createAdam
creating: createMaxEpoch
creating: createOptimizer
episode: 8/2000, score: 21, e: 0.98
creating: createMSECriterion
creating

KeyboardInterrupt: 

In [26]:
weights = np.array(agent.model.get_weights())

In [27]:
print(weights.shape)
for i in range(6):
    print(np.array(weights[i]).shape)

(6,)
(24, 4)
(24,)
(24, 24)
(24,)
(2, 24)
(2,)
